In [1]:
import azureml.core

print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.")

You are currently using version 1.38.0 of the Azure ML SDK.


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "NC6Cluster"

try:
    compute_target = ws.compute_targets[cluster_name]
    print("Found existing compute target.")
except KeyError:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_NC6",
        idle_seconds_before_scaledown=600,
        min_nodes=0,
        max_nodes=4,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(
    show_output=True, min_node_count=None, timeout_in_minutes=20
)

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.core import Experiment

experiment_name = "automl-image-multiclass-pneunomia"
experiment = Experiment(ws, name=experiment_name)

In [5]:
from azureml.core import Dataset
from azureml.data import DataType

# get existing training dataset
training_dataset_name = "pneumoniaTrainingDataset"
if training_dataset_name in ws.datasets:
    training_dataset = ws.datasets.get(training_dataset_name)
    print("Found the training dataset", training_dataset_name)

# get existing validation dataset
validation_dataset_name = "pneumoniaValidationDataset"
if validation_dataset_name in ws.datasets:
    validation_dataset = ws.datasets.get(validation_dataset_name)
    print("Found the validation dataset", validation_dataset_name)


Found the training dataset pneumoniaTrainingDataset
Found the validation dataset pneumoniaValidationDataset


In [6]:
training_dataset.to_pandas_dataframe()

,image_url,label
0,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
1,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
2,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
3,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
4,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
...,...,...
591,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
592,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
593,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
594,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA


In [ ]:
from azureml.automl.core.shared.constants import ImageTask
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import GridParameterSampling, choice

image_config_vit = AutoMLImageConfig(
    task=ImageTask.IMAGE_CLASSIFICATION,
    compute_target=compute_target,
    training_data=training_dataset,
    validation_data=validation_dataset,
    hyperparameter_sampling=GridParameterSampling({"model_name": choice("vitb16r224")}),
    iterations=1,
)